In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_6187/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_6187/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_21_8,0.953692,0.880261,0.915622,0.939461,0.018713,0.071082,0.080795,0.024074,0.075872,0.136794,0.938562,0.137819,73.957105,169.831173,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1,model_2_33_5,0.954705,0.879503,0.919463,0.964667,0.018303,0.071532,0.077116,0.013315,0.074757,0.135290,0.939906,0.136303,74.001349,169.875416,"Hidden Size=[8], regularizer=0.02, learning_ra..."
2,model_2_32_4,0.954261,0.879383,0.917026,0.938828,0.018483,0.071603,0.079450,0.022672,0.075856,0.135953,0.939316,0.136971,73.981797,169.855865,"Hidden Size=[8], regularizer=0.02, learning_ra..."
3,model_2_21_3,0.954453,0.879200,0.915061,0.948492,0.018405,0.071712,0.081332,0.020483,0.075997,0.135667,0.939571,0.136683,73.990216,169.864284,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_2_32_5,0.954403,0.879023,0.916855,0.939306,0.018425,0.071817,0.079614,0.022495,0.075542,0.135740,0.939505,0.136757,73.988052,169.862120,"Hidden Size=[8], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,model_4_16_3,0.974835,-16.875710,-13.478089,0.964982,0.010169,10.611793,13.863198,0.015663,0.050714,0.100841,0.908863,0.101596,203.176837,484.988491,"Hidden Size=[24], regularizer=0.02, learning_r..."
866,model_4_16_7,0.975591,-17.101318,-14.434469,0.963513,0.009863,10.745724,14.778959,0.016320,0.050723,0.099315,0.911601,0.100059,203.237841,485.049495,"Hidden Size=[24], regularizer=0.02, learning_r..."
867,model_4_14_4,0.937338,-17.439442,-16.254299,0.861992,0.025322,10.946449,16.521500,0.051862,0.058353,0.159127,0.773061,0.160319,201.352203,483.163856,"Hidden Size=[24], regularizer=0.02, learning_r..."
868,model_4_14_6,0.944075,-17.673135,-16.641635,0.872095,0.022599,11.085180,16.892386,0.048066,0.057124,0.150330,0.797459,0.151456,201.579683,483.391337,"Hidden Size=[24], regularizer=0.02, learning_r..."
